In [40]:
import sys
sys.path.append("../")
import data.directrepository as dr
import data.flight_repository as fr
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
import importlib as im
import queue
from enum import Enum
sys.path.append("../../")
import services.core.recoservice_df as reco
im.reload(reco)
from model.flight import Flight
from model.direct import Direct
import datetime


#im.reload(data.directrepository)
#im.reload(data.flight_repository)

In [41]:
f1 = Flight("JFK", "LHR", 1000, "DU", "", datetime.datetime(2019, 5, 10, 11, 0, 0),
            datetime.datetime(2019, 5, 10, 6, 0, 0), "123")
f2 = Flight("JFK", "MIA", 50, "DU", "", datetime.datetime(2019, 5, 5, 11, 0, 0),
            datetime.datetime(2019, 5, 5, 6, 0, 0), "456")
f3 = Flight("MIA", "LHR", 100, "DU", "", datetime.datetime(2019, 5, 8, 11, 0, 0),
            datetime.datetime(2019, 5, 8, 6, 0, 0), "456")
f4 = Flight("LHR", "EWR", 450, "DU", "", datetime.datetime(2019, 5, 18, 11, 0, 0),
            datetime.datetime(2019, 5, 18, 6, 0, 0), "123")
f5 = Flight("LHR", "ATL", 200, "DU", "", datetime.datetime(2019, 5, 18, 11, 0, 0),
            datetime.datetime(2019, 5, 18, 6, 0, 0), "123")
f6 = Flight("ATL", "JFK", 300, "DU", "", datetime.datetime(2019, 5, 20, 11, 0, 0),
            datetime.datetime(2019, 5, 20, 6, 0, 0), "123")
f7 = Flight("EWR", "LHR", 1200, "DU", "", datetime.datetime(2019, 5, 16, 11, 0, 0),
            datetime.datetime(2019, 5, 16, 6, 0, 0), "123")
f8 = Flight("JFK", "LHR", 1200, "DU", "", datetime.datetime(2019, 5, 16, 11, 0, 0), #dup of f1 but higher price
            datetime.datetime(2019, 5, 16, 6, 0, 0), "123")
f9 = Flight("JFK", "LHR", 5, "DU", "", datetime.datetime(2019, 5, 10, 11, 0, 0), #dup of f1 but on same day
            datetime.datetime(2019, 5, 10, 6, 0, 0), "123")
fs = [f1, f2, f3, f4, f5, f6, f7, f8,f9]
flights = pd.DataFrame.from_records([f.to_dict() for f in fs])
df = pd.DataFrame()

sources = ["JFK","EWR"]
targets = ["LHR"]
exclusions = []

df = pd.DataFrame()


In [42]:
result = reco.RecommendationService().get_recommendations(flights, sources, targets, exclusions,2,10)
result.to_json(orient='records')

'[{"current_loc":"EWR","current_path":[["DU",1557486000000,1557468000000,"","123","JFK","LHR",5,1],["DU",1558177200000,1558159200000,"","123","LHR","EWR",450,1]],"state":{"name":"at_completion"}},{"current_loc":"EWR","current_path":[["DU",1557054000000,1557036000000,"","456","JFK","MIA",50,1],["DU",1557313200000,1557295200000,"","456","MIA","LHR",100,1],["DU",1558177200000,1558159200000,"","123","LHR","EWR",450,1]],"state":{"name":"at_completion"}}]'

In [31]:
q = reco.RecommendationService().get_initial_queue(sources)
items = pd.DataFrame.from_records([f.to_dict() for f in list(q.queue)])

assert(len(items) == 2)
items

,current_loc,current_path,state
0,JFK,[],FlightState.at_source
1,EWR,[],FlightState.at_source


In [14]:

current = q.get()
if current.state == reco.FlightState.at_completion:
    df = df.append(current.to_dict(), ignore_index=True)
else:
    reco.RecommendationService().process(q, current, flights, exclusions, targets, sources, 2, 10)

items = pd.DataFrame.from_records([f.to_dict() for f in list(q.queue)])

items


""


In [15]:
#df = df.append(current.to_dict(),ignore_index=True)
df



,current_loc,current_path,state
0,EWR,"[[DU, 2019-05-10 11:00:00, 2019-05-10 06:00:00...",FlightState.at_completion
1,EWR,"[[DU, 2019-05-05 11:00:00, 2019-05-05 06:00:00...",FlightState.at_completion


In [33]:
#at target
fs = flights
prev_arrival_time = current.current_path[len(current.current_path)-1]["ArrivalTimeUTC"]
int_destinations = fs[fs["FlyFrom"].isin(targets)]["FlyTo"].unique()
f1 = fs[(fs["FlyFrom"] == current.current_loc) & 
        (fs["DepartTimeUTC"] > prev_arrival_time + datetime.timedelta(days=2)) & 
        (fs["DepartTimeUTC"] < prev_arrival_time + datetime.timedelta(days=8)) &
        (~fs["FlyTo"].isin(exclusions)) & 
        ((fs["FlyTo"].isin(int_destinations)) | (fs["FlyTo"].isin(sources)))]
f1 = f1.assign(rn = f1.sort_values(['Price']).groupby(['FlyTo']).cumcount()+1).query('rn < 2')




,Airline,ArrivalTimeUTC,DepartTimeUTC,Duration,FlightNum,FlyFrom,FlyTo,Price,rn
3,DU,2019-05-18 11:00:00,2019-05-18 06:00:00,,123,LHR,EWR,450,1
4,DU,2019-05-18 11:00:00,2019-05-18 06:00:00,,123,LHR,ATL,200,1


In [73]:
current.current_path[]


[Airline                            DU
 ArrivalTimeUTC    2019-05-05 11:00:00
 DepartTimeUTC     2019-05-05 06:00:00
 Duration                             
 FlightNum                         456
 FlyFrom                           JFK
 FlyTo                             MIA
 Price                              50
 rn                                  1
 Name: 1, dtype: object]

In [19]:
#at int1

prev_arrival_time = current.current_path[len(current.current_path)-1]["ArrivalTimeUTC"]
fs = flights
f1 = fs[(fs["FlyFrom"] == current.current_loc) & (fs["FlyTo"].isin(targets)) & 
        (fs["DepartTimeUTC"] > prev_arrival_time + datetime.timedelta(days=2)) & 
        (fs["DepartTimeUTC"] < prev_arrival_time + datetime.timedelta(days=3))]
f1 = f1.assign(rn = f1.sort_values(['Price']).groupby(['FlyTo']).cumcount()+1).query('rn < 2')

for index, first_stop in f1.iterrows():
    new_path = current.current_path.copy()
    new_path.append(first_stop)

    q.put(reco.FlightItem(first_stop["FlyTo"], new_path, reco.FlightState.at_target))
    
for elem in list(q.queue):
    print(str(elem))

AttributeError: 'FlightItem' object has no attribute 'print'